In [0]:
storageAccountKey=''
spark.conf.set("fs.azure.account.key.dmodemoaccount123.dfs.core.windows.net",storageAccountKey)

In [0]:
sourceCSVFilePath = 'abfss://working-labs@dmodemoaccount123.dfs.core.windows.net/bronze/daily-pricing/csv'
sourcePARQUETFilePath='abfss://working-labs@dmodemoaccount123.dfs.core.windows.net/bronze/daily-pricing/parquet'

In [0]:
sourcePARQUETFileDF=(spark
                     .read
                     .load(sourcePARQUETFilePath)
                     )

---------------------------------------------------------------------------
AnalysisException                         Traceback (most recent call last)
File <command-7747923595987401>, line 3
      1 sourcePARQUETFileDF=(spark
      2                      .read
----> 3                      .load(sourcePARQUETFilePath)
      4                      )

File /databricks/spark/python/pyspark/instrumentation_utils.py:47, in _wrap_function.<locals>.wrapper(*args, **kwargs)
     45 start = time.perf_counter()
     46 try:
---> 47     res = func(*args, **kwargs)
     48     logger.log_success(
     49         module_name, class_name, function_name, time.perf_counter() - start, signature
     50     )
     51     return res

File /databricks/spark/python/pyspark/sql/readwriter.py:312, in DataFrameReader.load(self, path, format, schema, **options)
    310 self.options(**options)
    311 if isinstance(path, str):
--> 312     return self._df(self._jreader.load(path))
    313 elif path is not None:


In [0]:
from pyspark.sql.functions import current_timestamp
sourcePARQUETTransFileDF=(sourcePARQUETFileDF
                          .withColumn("datalake_file_load_date",current_timestamp())
                          )

In [0]:
display(sourcePARQUETTransFileDF)

In [0]:
from pyspark.sql.functions import year,quarter
(sourcePARQUETTransFileDF
 .select("datalake_file_load_date")
 .withColumn("datalake_file_load_year",year("datalake_file_load_date"))
 .withColumn("datalake_file_load_quarter", quarter("datalake_file_load_date"))
 .show()
 )

In [0]:
from pyspark.sql.functions import month,dayofmonth
(sourcePARQUETTransFileDF
 .select("datalake_file_load_date")
 .withColumn("datalake_file_load_month",month("datalake_file_load_date"))
 .withColumn("datalake_file_load_day",dayofmonth("datalake_file_load_date"))
 .show()
 )


In [0]:
from pyspark.sql.functions import concat
from sklearn.model_selection import train_test_split
(sourcePARQUETTransFileDF
 .select("datalake_file_load_date")
 .withColumn("datalake_file_load_date_formatted",concat(year("datalake_file_load_date"),- month("datalake_file_load_date"),- dayofmonth("datalake_file_load_date")))
 .show()
 )  

In [0]:
from pyspark.sql.functions import date_format
(sourcePARQUETTransFileDF
 .select("datalake_file_load_date")
 .withColumn("datalake_file_load_date",date_format("datalake_file_load_date","yyyy-MM-dd"))
 .show()
)

In [0]:
sourcePARQUETTransFileDF.printSchema()

In [0]:
from pyspark.sql.functions import to_date
sourcePARQUETTransFileDF1=(sourcePARQUETTransFileDF
 .select("DATE_OF_PRICING")
 .withColumn("PRICING_DATE",to_date("DATE_OF_PRICING","dd/MM/yyyy")) 
)


      

In [0]:
(sourcePARQUETTransFileDF1
 .select("PRICING_DATE")
 .withColumn("PRICING_DATE_FROMATTED",date_format("PRICING_DATE","dd/MM/yyyy"))
 .show()
)